In [119]:
# ! pip install pandas
# ! pip install requests
# ! pip install plotly
# ! pip install datetime
# ! pip install os
# ! pip freeze = requirements.txt

In [120]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import numpy as np
import os

In [121]:
pwd = os.getcwd()
if 'L2 TVL' in pwd:
    prepend = ''
else:
    prepend = 'L2 TVL/'

In [122]:

api_str = 'https://api.llama.fi/protocol/'

protocols = [
    # name, incentive start date
         ['velodrome','2022-07-13']
        ,['pooltogether','2022-07-14']
        ,['lyra','2022-08-02']
        ,['rubicon','2022-07-15']
        ,['perpetual-protocol','2022-07-14']
        ,['thales','2022-07-14'] #TVL not relevant
        ,['aave-v3','2022-08-04']
        ,['wepiggy','2022-08-03']
        ,['stargate','2022-08-05']
        ]
# print(protocols[0])
prod = []
for prot in protocols:
    tp = r.get(api_str + prot[0]).json()['chainTvls']['Optimism']
    ad = pd.json_normalize( tp['tokens'] )
    ad_usd = pd.json_normalize( tp['tokensInUsd'] )
    if not ad.empty:
        ad = pd.melt(ad,id_vars = ['date'])
        ad = ad.rename(columns={'variable':'token','value':'token_value'})
        ad_usd = pd.melt(ad_usd,id_vars = ['date'])
        ad_usd = ad_usd.rename(columns={'variable':'token','value':'usd_value'})
        ad = ad.merge(ad_usd,on=['date','token'])
        
        ad['date'] = pd.to_datetime(ad['date'], unit ='s') #convert to days

        ad['token'] = ad['token'].str.replace('tokens.','', regex=False)
        ad['protocol'] = prot[0]
        ad['start_date'] = pd.to_datetime(prot[1])
        ad['date'] = ad['date'] - timedelta(days=1) #change to eod vs sod
        prod.append(ad)

df_df = pd.concat(prod)
df_df

,date,token,token_value,usd_value,protocol,start_date
0,2022-06-01 00:00:00,SETH,0.02061,3.740144e+01,velodrome,2022-07-13
1,2022-06-02 00:00:00,SETH,0.02061,3.786823e+01,velodrome,2022-07-13
2,2022-06-03 00:00:00,SETH,0.02609,4.651168e+01,velodrome,2022-07-13
3,2022-06-04 00:00:00,SETH,0.03786,6.849767e+01,velodrome,2022-07-13
4,2022-06-05 00:00:00,SETH,0.03142,5.697542e+01,velodrome,2022-07-13
...,...,...,...,...,...,...
275,2022-08-03 00:00:00,WETH,1881.40751,3.089779e+06,stargate,2022-08-05
276,2022-08-04 00:00:00,WETH,1518.29696,2.446371e+06,stargate,2022-08-05
277,2022-08-05 00:00:00,WETH,1326.15808,2.306799e+06,stargate,2022-08-05
278,2022-08-06 00:00:00,WETH,1007.90122,1.696026e+06,stargate,2022-08-05


In [123]:
# df_df[df_df['protocol']=='perpetual-protocol']


In [124]:
# #defillama api feedback - only token symbols come through, makes it hard to map w/o doing it manually
# coingecko_token_map = [
#     ['LINK','chainlink']
#     ,['USDT','tether']
#     ,['USDC','usd-coin']
#     ,['WETH','weth']
#     ,['SUSD','nusd']
#     ,['DAI','dai']
#     ,['AAVE','aave']
#     ,['WBTC','wrapped-bitcoin']
#     ,['SNX','havven']
#     ,['OP','optimism']
#     ,['SETH','seth']
#     ,['FXS','frax-share']
#     ,['THALES','thales']
#     ,['FRAX','frax']
#     ,['ALUSD','alchemix-usd']
#     ,['PERP','perpetual-protocol']
#     ,['LUSD','liquity-usd']
#     ,['LYRA','lyra-finance']
#     ,['HND','hundred-finance']
#     ,['BITANT','bitant']
#     ,['UNI','uniswap']
#     ,['SLINK','slink']
#     ,['VELO','velodrome-finance']
#     ,['DOLA','dola-usd']
#     ,['CRV','curve-dao-token']
#     ,['SBTC','sbtc']
#     ,['KROM','kromatika']
#     ,['DF','dforce-token']
#     ,['STG','stargate-finance']
#     ,['AELIN','aelin']
#     ,['RAI','rai']
#     ,['RETH','rocket-pool-eth']
# ]

In [125]:
# cg_token_list = [i[0] for i in coingecko_token_map]
# # print(cg_token_list)

In [126]:
# # DISTINCT TOKENS

# token_list = df_df[['token']].drop_duplicates()
# missing_token_list = token_list[~token_list['token'].isin(cg_token_list)]
# missing_token_list

In [127]:

# cg_pds = []
# for t in coingecko_token_map:
#     time.sleep(0.1)
#     cg_api = 'https://api.coingecko.com/api/v3/coins/'\
#             + t[1] + '/market_chart?vs_currency=usd&days=max&interval=daily'
#     print(cg_api)
#     pr = pd.DataFrame( r.get(cg_api).json()['prices'] )
#     pr['token'] = t[0]
#     pr['cg_slug'] = t[1]
#     pr = pr.rename(columns={0:'date',1:'price_usd'})
#     pr['date'] = pd.to_datetime(pr['date'], unit ='ms') #convert to days
#     cg_pds.append(pr)

# cg_df = pd.concat(cg_pds)
# # cg_df

In [128]:
data_df = df_df.copy()#merge(cg_df, on=['date','token'],how='inner')
data_df.sort_values(by='date',inplace=True)
data_df['token_value'] = data_df['token_value'].replace(0, np.nan)
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']

data_df.sort_values(by='date',inplace=True)

data_df['last_token_value'] = data_df.groupby(['token','protocol'])['token_value'].shift(1)
data_df['last_token_value'] = data_df['last_token_value'].fillna(0)

data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']

data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']


# display(data_df)

In [129]:
data_df[data_df['protocol']=='perpetual-protocol'].sort_values(by='date')


,date,token,token_value,usd_value,protocol,start_date,price_usd,last_token_value,net_token_flow,net_dollar_flow
0,2021-12-17 00:00:00,USDC,4.800012e+06,4.799268e+06,perpetual-protocol,2022-07-14,0.999845,0.000000e+00,4.800012e+06,4.799268e+06
1,2021-12-18 00:00:00,USDC,4.784074e+06,4.784074e+06,perpetual-protocol,2022-07-14,1.000000,4.800012e+06,-1.593859e+04,-1.593859e+04
2,2021-12-19 00:00:00,USDC,4.885148e+06,4.876546e+06,perpetual-protocol,2022-07-14,0.998239,4.784074e+06,1.010745e+05,1.008965e+05
3,2021-12-20 00:00:00,USDC,4.982993e+06,4.982993e+06,perpetual-protocol,2022-07-14,1.000000,4.885148e+06,9.784469e+04,9.784469e+04
4,2021-12-21 00:00:00,USDC,4.948937e+06,4.948937e+06,perpetual-protocol,2022-07-14,1.000000,4.982993e+06,-3.405556e+04,-3.405556e+04
...,...,...,...,...,...,...,...,...,...,...
229,2022-08-03 00:00:00,USDC,2.215844e+07,2.218059e+07,perpetual-protocol,2022-07-14,1.001000,2.142510e+07,7.333336e+05,7.340669e+05
230,2022-08-04 00:00:00,USDC,2.238048e+07,2.240286e+07,perpetual-protocol,2022-07-14,1.001000,2.215844e+07,2.220466e+05,2.222687e+05
231,2022-08-05 00:00:00,USDC,2.256065e+07,2.253182e+07,perpetual-protocol,2022-07-14,0.998722,2.238048e+07,1.801678e+05,1.799376e+05
232,2022-08-06 00:00:00,USDC,2.231180e+07,2.233411e+07,perpetual-protocol,2022-07-14,1.001000,2.256065e+07,-2.488518e+05,-2.491006e+05


In [130]:
netdf_df = data_df[data_df['date']>= data_df['start_date']][['date','protocol','net_dollar_flow']]
netdf_df = netdf_df.groupby(['date','protocol']).sum(['net_dollar_flow'])
netdf_df['cumul_net_dollar_flow'] = netdf_df.groupby(['protocol']).cumsum()
netdf_df.reset_index(inplace=True)

netdf_df

,date,protocol,net_dollar_flow,cumul_net_dollar_flow
0,2022-07-13 00:00:00,velodrome,1.273337e+06,1.273337e+06
1,2022-07-14 00:00:00,perpetual-protocol,8.596476e+04,8.596476e+04
2,2022-07-14 00:00:00,pooltogether,-1.567899e+04,-1.567899e+04
3,2022-07-14 00:00:00,thales,9.680946e+03,9.680946e+03
4,2022-07-14 00:00:00,velodrome,5.143013e+06,6.416351e+06
...,...,...,...,...
138,2022-08-06 16:59:12,lyra,9.919125e+05,1.015625e+07
139,2022-08-06 16:59:12,pooltogether,1.903464e+05,3.031517e+07
140,2022-08-06 16:59:12,stargate,-4.982817e+05,9.300827e+06
141,2022-08-06 16:59:12,velodrome,-1.832429e+06,8.794166e+07


In [131]:
fig = px.line(netdf_df, x="date", y="net_dollar_flow", color="protocol", \
             title="Daily Net Dollar Flow since Program Announcement",\
            labels={
                     "date": "Day",
                     "net_dollar_flow": "Net Dollar Flow (N$F)"
                 }
            )
fig.update_layout(
    legend_title="App Name"
)
fig.update_layout(yaxis_tickprefix = '$')
fig.write_image(prepend + "img_outputs/svg/daily_ndf.svg")
fig.write_image(prepend + "img_outputs/png/daily_ndf.png")
fig.write_html(prepend + "img_outputs/daily_ndf.html", include_plotlyjs='cdn')

# cumul_fig = px.area(netdf_df, x="date", y="cumul_net_dollar_flow", color="protocol", \
#              title="Cumulative Dollar Flow since Program Announcement",\
#                    labels={
#                      "date": "Day",
#                      "cumul_net_dollar_flow": "Cumulative Net Dollar Flow (N$F)"
#                  }
#             ,areamode='group')
# cumul_fig.update_layout(yaxis_tickprefix = '$')
# cumul_fig.show()


cumul_fig = go.Figure()
proto_names = netdf_df['protocol'].drop_duplicates()
print(proto_names)
for p in proto_names:
    cumul_fig.add_trace(go.Scatter(x=netdf_df[netdf_df['protocol'] == p]['date'] \
                                   , y=netdf_df[netdf_df['protocol'] == p]['cumul_net_dollar_flow'] \
                                    ,name = p\
                                  ,fill='tozeroy')) # fill down to xaxis

cumul_fig.update_layout(yaxis_tickprefix = '$')
cumul_fig.update_layout(
    title="Cumulative Dollar Flow since Program Announcement",
    xaxis_title="Day",
    yaxis_title="Cumulative Net Dollar Flow (N$F)",
    legend_title="App Name",
#     color_discrete_map=px.colors.qualitative.G10
)
cumul_fig.write_image(prepend + "img_outputs/svg/cumul_ndf.svg") #prepend + 
cumul_fig.write_image(prepend + "img_outputs/png/cumul_ndf.png") #prepend + 
cumul_fig.write_html(prepend + "img_outputs/cumul_ndf.html", include_plotlyjs='cdn')


0               velodrome
1      perpetual-protocol
2            pooltogether
3                  thales
7                 rubicon
95                   lyra
107               wepiggy
108               aave-v3
121              stargate
Name: protocol, dtype: object


In [132]:
# fig.show()
# cumul_fig.show()
print("yay")

yay


In [137]:
# ! jupyter nbconvert --to python optimism_app_net_flows.ipynb

[NbConvertApp] Converting notebook optimism_app_net_flows.ipynb to python
[NbConvertApp] Writing 6970 bytes to optimism_app_net_flows.py
